In [7]:
import sys
sys.path.insert(0, '../..')
from helper import _process_pred_labels
from evaluate import get_pred_labels

In [1]:
import os
import pandas as pd
import numpy as np

In [2]:
from sklearn.linear_model import LogisticRegression, ElasticNet
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import cross_validate, GridSearchCV
from sklearn.svm import LinearSVC, SVC
from sklearn.calibration import CalibratedClassifierCV
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier

In [3]:
N_FILES = 2
WEIGHTS = (0.791782261928, 0.78721951357)
SUM_WEIGHTS = sum(WEIGHTS)

In [4]:
assert N_FILES == len(WEIGHTS)

In [5]:
pred_proba = pd.read_csv(f'sub_0/pred_proba.csv', index_col='review_id')
for col in pred_proba.columns:
    pred_proba[col] = 0

for i, w in zip(range(N_FILES), WEIGHTS):
    df = pd.read_csv(f'sub_{i}/pred_proba.csv', index_col='review_id')
    # df = df.rank(ascending=False) / len(df)
    df *= (w / SUM_WEIGHTS)
    pred_proba += df

In [8]:
pred_proba = pd.DataFrame(
    data=get_pred_labels(pred_proba.values),
    index=pred_proba.index,
    columns=pred_proba.columns
)
pred_proba['target'] = pred_proba.apply(_process_pred_labels, axis=1)

In [9]:
pred_proba['target'].to_csv('blending.csv')